# バッチ正規化・Layer正規化・Instance正規化（CIFAR10を用いた物体認識）

---
## 目的
畳み込みニューラルネットワーク (Convolutional Neural Network; CNN) を用いてCIFAR10データセットに対する物体認識を行う．
その際，様々な正規化を用いることで，認識性能がどのように変化するかを確認する．

## 対応するチャプター
* 8.7.1: バッチ正規化
* 8.1.3: バッチアルゴリズムとミニバッチアルゴリズム
* 8.3.1: 確率的勾配降下法
* 9.1: 畳み込み処理
* 9.3: プーリング


## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torchvision.datasets import CIFAR10
from torchvision import transforms

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込み
CIFAR10データセットを読み込みます．

読み込んだ学習データのサイズを確認します．
学習データは5万枚，1つのデータサイズは3x32x32の画像のような形式となっています．
これは32x32ピクセルのカラー画像という意味になります．

In [ ]:
train_data = CIFAR10(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = CIFAR10(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(train_data)
print(test_data)

## ネットワークモデルの定義
畳み込みニューラルネットワークを定義します．

ここでは，畳み込み層2層，全結合層3層から構成されるネットワークとします．

このネットワークでは，畳み込み層を適用した後に正規化を行います．

`__init__()`の引数として，`norm`を選択し，使用する正規化の種類を選択します．
そして，各畳み込み層に対する正規化を`self.norm1`, `self.norm2`として定義を行います．
バッチ正規化（`BatchNorm2d`）およびインスタンス正規化（`InstanceNorm2d`）では，正規化する特徴の次元数（チャンネル数）を設定します．
またLayer正規化（`LayerNorm`）では，入力される特徴マップのサイズを設定します．


推論時は，畳み込み --> 正規化 --> 活性化関数 --> (必要に応じてPooling) の順番で処理を行います．

In [ ]:
class CNN(nn.Module):
    def __init__(self, norm='batch'):
        super().__init__()
        assert norm in ['batch', 'layer', 'instance'], "norm should be selected from 'batch', 'layer', 'instance'"
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        
        if norm == 'batch':
            self.norm1 = nn.BatchNorm2d(16)
        elif norm == 'layer':
            self.norm1 = nn.LayerNorm((16, 32, 32))
        elif norm == 'instance':
            self.norm1 = nn.InstanceNorm2d(16)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        if norm == 'batch':
            self.norm2 = nn.BatchNorm2d(32)
        elif norm == 'layer':
            self.norm2 = nn.LayerNorm((32, 16, 16))
        elif norm == 'instance':
            self.norm2 = nn.InstanceNorm2d(32)
        
        self.l1 = nn.Linear(8 * 8 * 32, 1024)
        self.l2 = nn.Linear(1024, 1024)
        self.l3 = nn.Linear(1024, 10)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        h = self.pool(self.act(self.norm1(self.conv1(x))))
        h = self.pool(self.act(self.norm2(self.conv2(h))))
        h = h.view(h.size()[0], -1)
        h = self.act(self.l1(h))
        h = self.act(self.l2(h))
        h = self.l3(h)
        return h

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．

ここでは，使用する正規化の種類を`norm_type`に設定し，ネットワークモデルを作成します．

In [ ]:
norm_type = 'instance'

model = CNN(norm=norm_type)
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を64，学習エポック数を20とします．
CIFAR10の学習データサイズを取得し，1エポック内における更新回数を求めます．
学習データは毎エポックでランダムに利用するため，numpyの`permutation`という関数を利用します．
各更新において，学習用データと教師データをそれぞれ`x`と`t`とし，`to_gpu`関数でGPUに転送します．
学習モデルにxを与えて各クラスの確率`y`を取得します．
各クラスの確率`y`と教師ラベル`t`との誤差を`softmax_coross_entropy`誤差関数で算出します．
また，認識精度も算出します．
そして，誤差を`backward`関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)
        
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))

## テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## 課題
1. 正規化の有無や種類の違いによる認識性能の変化を確認しましょう
